In [ ]:
#                                               ::  GAIA  ::

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, Reshape
from tensorflow.keras.optimizers import Adam

# Load data 
file_path = r'C:\Users\Subhadeep Maishal\Music\JUP\test_data\CBN_INPUT.csv'
df = pd.read_csv(file_path, parse_dates=['DATE'])
features = df[['AAO', 'AO', 'DMI', 'NAO', 'ONI', 'PDO', 'SOI', 'TNA', 'TSA', 'WHWP', 'WP']]
target = df['MEIv2']
features = (features - features.min()) / (features.max() - features.min())
np.random.seed(100)


#----------------------------------------------start GAIA------------------------------------------------------------------ 

# CNN model
def build_cnn(input_shape):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='linear', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(128, kernel_size=3, activation='linear'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='linear'))
    return model

# RNN model
def build_rnn(input_shape):
    model = Sequential()
    model.add(LSTM(4000, input_shape=input_shape, return_sequences=True))
    model.add(LSTM(4000, return_sequences=True))
    model.add(LSTM(4000))
    model.add(Dense(1, activation='linear'))
    return model

# Combined model with CNN and RNN
def build_combined_model(cnn, rnn, input_shape):
    model = Sequential()
    model.add(cnn)
    model.add(Reshape((1, -1)))  # Flatten the output of CNN
    model.add(LSTM(3000, return_sequences=True))  # Adjust dimensions based on the number of features
    model.add(LSTM(3000, return_sequences=True))
    model.add(LSTM(3000))
    model.add(Dense(1, activation='linear'))
    return model


#-------------------------------------------------end GAIA------------------------------------------------------------------ 


# Build and compile the CNN
cnn = build_cnn((features.shape[1], 1))
cnn.compile(loss='mean_squared_error', optimizer=Adam(0.0005, 0.05))

# Build and compile the RNN
rnn = build_rnn((features.shape[1], 1))
rnn.compile(loss='mean_squared_error', optimizer=Adam(0.0005, 0.05))




# Build the combined model (CNN-RNN)
combined_model = build_combined_model(cnn, rnn, (features.shape[1], 1))
combined_model.compile(loss='mean_squared_error', optimizer=Adam(0.0005, 0.05))





# Training the combined model (adjust as needed)
epochs = 120
batch_size = 32

for epoch in range(epochs):
    idx = np.random.randint(0, features.shape[0], batch_size)
    real_data = features.iloc[idx].values  # Convert to numpy array
    real_labels = target.iloc[idx].values

    # Train the combined model
    combined_loss = combined_model.train_on_batch(real_data, real_labels)

    if epoch % 10 == 0:
        print(f"{epoch} [Combined loss: {combined_loss}]")


        
        
# Generate data using RNN >>
num_samples = 420
noise = np.random.normal(0, 1, (num_samples, features.shape[1], 1))
synthetic_data = combined_model.predict(noise)







# Plot actual and generated data
plt.figure(figsize=(12, 8))
plt.plot(df['DATE'], target, label='Actual', marker='o', color='black')
plt.plot(df['DATE'].iloc[-num_samples:], synthetic_data[:, 0], label='Generated', marker='o', color='blue')
plt.xlabel('Date', fontsize=14)
plt.ylabel('MEIv2 Values', fontsize=14)
plt.title('Actual vs  Generated MEIv2 Values Over Time', fontsize=16)
plt.legend(fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
#                                       ::   Cheers!  ::

In [ ]:
#                                       ::  Subhadeep  ::